In [24]:
!pip install nltk evaluate
!pip install rouge-score
!pip install absl-py


In [25]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import evaluate



In [26]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')  # For tokenization


[nltk_data] Downloading package wordnet to /Users/aniketh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aniketh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
results_df = pd.read_csv('results_1.csv')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess text by lemmatizing each word
def preprocess(text):
    words = word_tokenize(text.lower())
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

results_df['Generated_tokens'] = results_df['Generated'].apply(preprocess)
results_df['Expected_tokens'] = results_df['Expected'].apply(preprocess)

# Calculate BLEU scores
results_df['BLEU'] = results_df.apply(lambda row: sentence_bleu([row['Expected_tokens']], row['Generated_tokens'], weights=(0.25, 0.25, 0.25, 0.25)) if row['Expected_tokens'] and row['Generated_tokens'] else 0, axis=1)
average_bleu = results_df['BLEU'].mean()

# Calculate ROUGE scores 
rouge_evaluator = evaluate.load('rouge')
rouge_results = rouge_evaluator.compute(predictions=results_df['Generated'], references=results_df['Expected'], use_stemmer=False)

# Output the results
print("Results:")
print(results_df.head())
print(f"Average BLEU score: {average_bleu}")
print("ROUGE Scores:", rouge_results)

#results_df.to_csv('modified_results.csv', index=False)

/Users/aniketh/anaconda3/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/aniketh/anaconda3/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/aniketh/anaconda3/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

Results:
   Unnamed: 0                                          Generated  \
0           0  "This is just the start we needed," Dancer sai...   
1           1   opposes, planning, are, supported, accuses, v...   
2           2   introduced, rein, profits, investment, smelti...   
3           3  2005, was, used, captured, access, scenes, att...   
4           4   said, said, was, ruling, called, was, said, c...   

                               Expected  \
0                     win, title, count   
1                               accuses   
2                       introduce, rein   
3  made, used, capture, attacks, murder   
4                                  said   

                                    Generated_tokens  \
0  [``, this, is, just, the, start, we, needed, ,...   
1  [opposes, ,, planning, ,, are, ,, supported, ,...   
2  [introduced, ,, rein, ,, profit, ,, investment...   
3  [2005, ,, wa, ,, used, ,, captured, ,, access,...   
4  [said, ,, said, ,, wa, ,, ruling, ,, cal

In [17]:
#Accuracy Method 1: checks if all the expected keywords are present in the generated text and calculates 
#an accuracy score based on this.

def keywords_present(generated, expected):
    expected_keywords = {word.strip().lower() for word in expected.split(',')}
    generated_text = generated.lower()
    return all(keyword in generated_text for keyword in expected_keywords)

results_df['Contains_All_Keywords'] = results_df.apply(lambda row: all_keywords_present(row['Generated'], row['Expected']), axis=1)

accuracy = results_df['Contains_All_Keywords'].mean()

print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.78


In [19]:
#Accuracy Method 2: calculates the proportion of expected keywords found in the generated text for each pair of 
#expected and generated texts and then computes the average of these proportions.

results_df = pd.read_csv('results_1.csv')

accuracies = []

for (generated, expected) in zip(results_df["Generated"], results_df["Expected"]):
  expected_list = expected.lower().split(",")
  accuracies.append(sum(word in generated.lower() for word in expected_list) / len(expected_list))

print(np.mean(accuracies))

0.8822950761975152
